In [69]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Sklearn
import sklearn.preprocessing
import sklearn.metrics
import sklearn.feature_selection
import sklearn.model_selection

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Import Data

In [70]:
train_identity = pd.read_csv('/Users/zhaoruidi/Documents/Kaggle/Data/train_identity.csv')
test_identity = pd.read_csv('/Users/zhaoruidi/Documents/Kaggle/Data/test_identity.csv')
train_transaction = pd.read_csv('/Users/zhaoruidi/Documents/Kaggle/Data/train_transaction.csv')
test_transaction = pd.read_csv('/Users/zhaoruidi/Documents/Kaggle/Data/test_transaction.csv')

test_identity.columns = [i.replace('-', '_') for i in test_identity.columns]

In [71]:
# 合并train and test
transaction = train_transaction.append(test_transaction, ignore_index=True)
identity = train_identity.append(test_identity, ignore_index=True)
transaction_x = transaction.drop('isFraud', axis = 1)
transaction_y = transaction['isFraud']

In [72]:
transaction_x.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


## Merge `transaction_x` and `identity`

In [ ]:
# 合并transaction and identity
# 注意包括了train & test
X_train = transaction_x.merge(identity, how = 'left', on = 'TransactionID')
X_train

# Data Preprocessing

## Missingness

### 算一下missingness rate

In [ ]:
missing = (X_train.isnull().sum()/X_train.shape[0] * 100).to_frame()
missing.columns = ['% Missing']
missing = missing[missing['% Missing'] != 0]

missing.sort_values('% Missing', ascending = False).head()

### 分一下Categorical/Numerical Features

In [ ]:
# Get the categorical columns
cat_col = (
    ['ProductCD'] + ['card' + str(i) for i in range(1, 7)] + ['addr1', 'addr2'] 
    + ['P_emaildomain'] + ['R_emaildomain'] + ['M' + str(i) for i in range(1, 10)] + ['DeviceType','DeviceInfo']
    + ['id_' + str(i) for i in range(12, 39)]
)

# Get the numerical columns
all_col = set(X_train.columns.tolist())
num_col = all_col - set(cat_col) - set(['TransactionID'])
num_col = list(num_col)

print('Total number of columns:', len(X_train.columns))
print('Number of categorical columns', len(cat_col))
print('Number of numerical columns', len(num_col))

### Impute data (Categorical)

In [ ]:
X_train_cat = X_train[cat_col]
X_train_cat

- 超过90%没有的先扔了
- 需要用mode的
    - `card2` to `card6`
    - `addr1` and `addr2`
- 直接写missing
    - `P_emaildomain` and `R_emaildomain`
    - `M1` to `M9`, 因为都是T/F
    - `DeviceType` and `DeviceInfo`
    - 剩下所有
    

In [ ]:
cat_missing = X_train_cat.isnull().mean()
cat_drop =  cat_missing[cat_missing > 0.9].index.tolist()
print('Number of over 90% missingness categorical column:', len(cat_drop))

# 大于90%的col实在处理不了，drop了
X_train.drop(cat_drop, axis = 1, inplace = True)
X_train.head()

#### Impute with mode

In [ ]:
mode_col = ['card2', 'card6', 'addr1', 'addr2']
for i in mode_col:
    X_train[i] = X_train.groupby(['ProductCD'])[i].apply(lambda x:x.fillna(x.value_counts().index[0]))

In [ ]:
X_train[mode_col].isnull().mean()
# Everything clear!

#### Impute with "missing"

In [ ]:
# missing当做一个单独的category来看？

In [ ]:
cat_drop

In [ ]:
for i in (set(cat_col) - set(cat_drop)) :
    X_train[i].fillna('missing', inplace = True)

### Impute date (Numerical)

In [ ]:
X_train_num = X_train[num_col]
X_train_num

In [ ]:
X_train_num.isnull().mean()

In [ ]:
num_missing = X_train_num.isnull().mean()
num_drop =  num_missing[num_missing > 0.6].index.tolist()
print('Number of over 90% missingness numerical column:', len(num_drop))

# 大于90%的col实在处理不了，drop了
X_train.drop(num_drop, axis = 1, inplace = True)
X_train.head()

In [64]:
num_drop

['id_08', 'id_07', 'dist2', 'D7']

In [48]:
# X_train_identity = train_identity[['TransactionID', 'id_01', 'id_12']]
X_train = train_transaction[['TransactionID', 'TransactionAmt', 'ProductCD', 'isFraud']]

# X_train = X_train_identity.merge(X_train_transaction, how = 'left', on = 'TransactionID')

y_train = X_train['isFraud']
X_train = X_train.drop('isFraud', axis = 1)

X_train.drop('TransactionID', axis = 1, inplace = True)
X_train = pd.get_dummies(X_train, ['ProductCD'], drop_first=True)

X_train

,TransactionAmt,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W
0,68.50,0,0,0,1
1,29.00,0,0,0,1
2,59.00,0,0,0,1
3,50.00,0,0,0,1
4,50.00,1,0,0,0
...,...,...,...,...,...
590535,49.00,0,0,0,1
590536,39.50,0,0,0,1
590537,30.95,0,0,0,1
590538,117.00,0,0,0,1


In [24]:
# Missingness rate of transaction
missing_transaction = (transaction_x.isnull().sum()/transaction_x.shape[0] * 100).to_frame()
missing_transaction.columns = ['% Missing']
missing_transaction = missing_transaction[missing_transaction['% Missing'] != 0]

# 把missingness rate < 10%的column选出来
transaction_col = missing_transaction[missing_transaction['% Missing'] < 10].index.tolist()

# Missingness rate of identity
missing_identity = (identity.isnull().sum()/identity.shape[0] * 100).to_frame()
missing_identity.columns = ['% Missing']
missing_identity = missing_identity[missing_identity['% Missing'] != 0]
missing_identity.head()

# 把missingness rate < 10%的column选出来
identity_col = missing_identity[missing_identity['% Missing'] < 10].index.tolist()

# 两个合并一下，这就是我暂时想focus的columns
all_col = transaction_col + identity_col
print('The number of columns left:', len(all_col))

The number of columns left: 190


In [50]:
# X_test_identity = test_identity[['TransactionID', 'id_01', 'id_12']]
X_test = test_transaction[['TransactionID', 'TransactionAmt', 'ProductCD']]

# X_test = X_test_identity.merge(X_test_transaction, how = 'left', on = 'TransactionID')

X_test.drop('TransactionID', axis = 1, inplace = True)
X_test = pd.get_dummies(X_test, ['ProductCD'], drop_first=True)

X_test

,TransactionAmt,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W
0,31.950,0,0,0,1
1,49.000,0,0,0,1
2,171.000,0,0,0,1
3,284.950,0,0,0,1
4,67.950,0,0,0,1
...,...,...,...,...,...
506686,94.679,0,0,0,0
506687,12.173,0,0,0,0
506688,49.000,0,0,0,1
506689,202.000,0,0,0,1


# Train the model

In [51]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier()
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [52]:
prediction = xgb_clf.predict(X_test)

In [53]:
result = pd.DataFrame({'TransactionID':test_transaction['TransactionID'], 'isFraud':prediction})
result.head()

,TransactionID,isFraud
0,3663549,0
1,3663550,0
2,3663551,0
3,3663552,0
4,3663553,0


In [54]:
result.to_csv('result.csv', index = False)